### When I run vanilla MD with run_md_hybrid_read_htf.py and save positions, all positions are 0 starting at around 2000 iterations -- figure out why

In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import LangevinIntegrator
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from simtk import openmm, unit
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys


INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [8]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.DEBUG)



In [3]:
# Define simulation parameters
temperature = 300 * unit.kelvin
collision_rate = 1.0 / unit.picoseconds
nsteps = 5000000 # 20 ns
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/7/45/"
phase = 'solvent'
endstate = 0



In [4]:
i = os.path.basename(os.path.dirname(out_dir))
htf = pickle.load(open(os.path.join(out_dir, f"{i}_{phase}.pickle"), "rb" ))


system = htf.hybrid_system
positions = htf.hybrid_positions


In [5]:
# Create CompoundThermodynamicState at the appropriate endstate
lambda_alchemical_state = RelativeAlchemicalState.from_system(system)
lambda_protocol = LambdaProtocol(functions = 'default')
lambda_alchemical_state.set_alchemical_parameters(endstate, lambda_protocol)
thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(system, temperature=temperature), composable_states=[lambda_alchemical_state])

# Set up integrator
integrator = LangevinIntegrator(temperature, collision_rate, timestep)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = thermodynamic_state.create_context(integrator, platform=platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)


In [6]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [9]:
# Run equilibration
final_pos = np.empty(shape=(2501, htf.hybrid_topology.n_atoms, 3))
pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
i = 0
final_pos[i] = pos * unit.nanometers
for step in tqdm(range(nsteps)):
    initial_time = time.time()
    integrator.step(1)
    if step % 2000 == 0:
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        final_pos[i] = pos *unit.nanometers
        i += 1
        elapsed_time = (time.time() - initial_time) * unit.seconds
        _logger.info(f'Step: {step} took {elapsed_time} seconds')



  3%|▎         | 139894/5000000 [01:11<41:22, 1957.54it/s]INFO:root:Step: 140000 took 0.0016260147094726562 s seconds


  6%|▌         | 279804/5000000 [02:22<40:02, 1964.87it/s]INFO:root:Step: 280000 took 0.0017342567443847656 s seconds


  8%|▊         | 419877/5000000 [03:35<39:09, 1949.60it/s]INFO:root:Step: 420000 took 0.0016074180603027344 s seconds


 11%|█         | 559808/5000000 [04:47<38:21, 1929.15it/s]INFO:root:Step: 560000 took 0.0013031959533691406 s seconds


 14%|█▍        | 699912/5000000 [06:00<36:59, 1937.83it/s]INFO:root:Step: 700000 took 0.0016674995422363281 s seconds


...

 28%|██▊       | 1389942/5000000 [11:59<31:07, 1932.64it/s]INFO:root:Step: 1390000 took 0.0014576911926269531 s seconds


 31%|███       | 1525827/5000000 [13:10<29:45, 1945.94it/s]INFO:root:Step: 1526000 took 0.0014483928680419922 s seconds


 33%|███▎      | 1661849/5000000 [14:20<28:52, 1927.27it/s]INFO:root:Step: 1662000 took 0.0015323162078857422 s seconds


 33%|███▎      | 1668161/5000000 [14:23<28:44, 1931.64it/s]


KeyboardInterrupt: 

In [ ]:
# Save traj
i = os.path.basename(os.path.dirname(args.dir))
with open(os.path.join(args.dir, f"{i}_{args.phase}_equil_hybrid_{args.endstate}_20ns.npy"), 'wb') as f:
    np.save(f, final_pos)
